In [1]:
import mido
import os
import numpy as np
from scipy import stats
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from tensorflow.contrib import rnn
import time

import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = (12, 8)

In [2]:
TICKS_PER_BEAT = 480
TEMPO = int(mido.bpm2tempo(120))

In [3]:
%run get_training_data.py

### Create training data

In [4]:
def get_training_data(num):
    """Generate training data array for all files in "midis_processed/" directory.
       
    Returns:
        Numpy array of training data.
    """
    training_data = []
    for filename in os.listdir('midis_processed/')[:num]:
        if filename.endswith(".mid"):
            vector = midi_to_vector('midis_processed/' + filename)
            if len(vector) > 100:
                training_data.append(vector)
            else:
                print("Faulty training data: " + filename + ". len(vector)=" + str(len(vector)))
    return np.array(training_data)

In [5]:
training_data = get_training_data(300)

Faulty training data: Adventure_Island_II-Eggplant.mid. len(vector)=0
Faulty training data: Adventure_Island_II-Game_Over.mid. len(vector)=0
Faulty training data: Adventure_Island_II-Victory.mid. len(vector)=0
Faulty training data: Alfred_Chicken_Level_1.mid. len(vector)=0
Faulty training data: alien3sb.mid. len(vector)=0
Faulty training data: alphago.mid. len(vector)=0
Faulty training data: Amagon-Death.mid. len(vector)=0
Faulty training data: Amagon-Start_2.mid. len(vector)=0


In [6]:
training_data = np.array([np.array(x) for x in training_data])
training_data

array([ array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0,

### Parameters

In [7]:
vocab_size = len(training_data[0][0])
vocab_size

1124

In [8]:
batch_size = 50
batch_size

92

In [ ]:
# Parameters
learning_rate = 0.003
training_iters = 10000
display_step = 10
n_input = 100

# number of units in RNN cell
n_hidden = 512

In [9]:
# tf Graph input
x = tf.placeholder("float", [batch_size, n_input, vocab_size])
y = tf.placeholder("int32", [batch_size, n_input])

# RNN output node weights and biases
softmax_w = tf.Variable(tf.random_normal([n_hidden, vocab_size]))
softmax_b = tf.Variable(tf.random_normal([vocab_size]))

max_grad_norm = 5

In [10]:
# tf Graph generative input
x_generation = tf.placeholder("float", [1, vocab_size])

## Create tensor flow graph

In [11]:
rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden), rnn.BasicLSTMCell(n_hidden)])

In [185]:
x_split = tf.split(x, n_input, 1)

In [186]:
x_split = [tf.reshape(a, [batch_size, vocab_size]) for a in x_split]

In [14]:
outputs, states = rnn.static_rnn(rnn_cell, x_split, dtype=tf.float32)
outputs

[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_2:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_5:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_8:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_11:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_14:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_17:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_20:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_23:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_26:0' shape=(92, 512) dtype=float32>,
 <tf.Tensor 'rnn/rnn/multi_rnn_

In [15]:
logits = [tf.nn.xw_plus_b(output, softmax_w, softmax_b) for output in outputs]
logits

[<tf.Tensor 'xw_plus_b:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_1:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_2:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_3:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_4:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_5:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_6:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_7:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_8:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_9:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_10:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_11:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_12:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_13:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_14:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus_b_15:0' shape=(92, 1124) dtype=float32>,
 <tf.Tensor 'xw_plus

In [16]:
logits = tf.reshape(logits, [batch_size, n_input, vocab_size])
logits

<tf.Tensor 'Reshape_50:0' shape=(92, 50, 1124) dtype=float32>

In [17]:
loss = tf.contrib.seq2seq.sequence_loss(logits, y, weights=tf.ones([batch_size, n_input], dtype=tf.float32), # check this
                                        average_across_timesteps=True, average_across_batch=True)
loss

<tf.Tensor 'sequence_loss/truediv:0' shape=(50,) dtype=float32>

In [18]:
cost = tf.reduce_sum(loss)
cost

<tf.Tensor 'Sum:0' shape=() dtype=float32>

In [19]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer

<tf.Operation 'RMSProp' type=NoOp>

## Run tensor flow session

In [20]:
session = tf.Session()

In [21]:
def get_midi_xy(midi_file_number):
    offset = np.random.randint(0, len(training_data[midi_file_number]) - n_input - 1)
    midi_x = training_data[midi_file_number][offset:offset+n_input]
    midi_y = training_data[midi_file_number][1 + offset:1 + offset+n_input].dot(range(vocab_size)).reshape(1, n_input)
    return midi_x, midi_y

In [22]:
def get_batch():
    """Generate random batches of x and y training vectors for RNN.
       
    Returns:
        Two batches of x and y vectors.
    """
    batch_x = []
    batch_y = []
    batch = np.random.choice(range(len(training_data)), batch_size, replace=False)
    for b in batch:
        midi_x, midi_y = get_midi_xy(b%len(training_data))
        batch_x.append(midi_x)
        batch_y.append(midi_y)
        batch_y = np.reshape(batch_y, [batch_size, n_input])
    return batch_x, batch_y

In [23]:
def seconds_to_minsecstr(seconds):
    minutes = seconds//60
    seconds -= 60*minutes
    if minutes < 10: minutes = "0" + str(minutes)
    if seconds < 10: seconds = "0" + str(seconds)
    return str(minutes) + ":" + str(seconds)

In [24]:
training_loss_list = []

init = tf.global_variables_initializer()
session.run(init)
step = 0
loss_total = 0

start_time = time.time()

while step < training_iters:
    batch_x, batch_y = get_batch()

    _, loss = session.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
    loss_total += loss
    if (step+1) % display_step == 0:
        print("Iter=" + str(step+1) + ", Average Loss={:.6f}, Time Elapsed={}".format(loss_total/display_step, 
             seconds_to_minsecstr(int(time.time()-start_time))))
        loss_total = 0
    training_loss_list.append(loss_total/display_step)
    step += 1

Iter=1, Average Loss=376.194122, Time Elapsed=00:00.07
Iter=2, Average Loss=362.741852, Time Elapsed=00:00.14
Iter=3, Average Loss=354.501862, Time Elapsed=00:00.20
Iter=4, Average Loss=349.700958, Time Elapsed=00:00.26
Iter=5, Average Loss=341.575897, Time Elapsed=00:00.34
Iter=6, Average Loss=339.359161, Time Elapsed=00:00.42
Iter=7, Average Loss=333.342194, Time Elapsed=00:00.49
Iter=8, Average Loss=335.661804, Time Elapsed=00:00.56
Iter=9, Average Loss=334.497803, Time Elapsed=00:01.03
Iter=10, Average Loss=331.893250, Time Elapsed=00:01.09
Iter=11, Average Loss=343.635162, Time Elapsed=00:01.15
Iter=12, Average Loss=331.490845, Time Elapsed=00:01.21
Iter=13, Average Loss=323.286560, Time Elapsed=00:01.27
Iter=14, Average Loss=322.483215, Time Elapsed=00:01.34
Iter=15, Average Loss=329.100159, Time Elapsed=00:01.40
Iter=16, Average Loss=357.856598, Time Elapsed=00:01.47
Iter=17, Average Loss=344.309082, Time Elapsed=00:01.53
Iter=18, Average Loss=329.769836, Time Elapsed=00:01.59
I

Iter=147, Average Loss=271.929382, Time Elapsed=00:15.07
Iter=148, Average Loss=267.983215, Time Elapsed=00:15.11
Iter=149, Average Loss=270.830139, Time Elapsed=00:15.16
Iter=150, Average Loss=261.905060, Time Elapsed=00:15.20
Iter=151, Average Loss=260.138184, Time Elapsed=00:15.25
Iter=152, Average Loss=267.838684, Time Elapsed=00:15.30
Iter=153, Average Loss=268.758636, Time Elapsed=00:15.34
Iter=154, Average Loss=265.636505, Time Elapsed=00:15.39
Iter=155, Average Loss=269.743347, Time Elapsed=00:15.43
Iter=156, Average Loss=267.147278, Time Elapsed=00:15.47
Iter=157, Average Loss=262.826935, Time Elapsed=00:15.52
Iter=158, Average Loss=263.173523, Time Elapsed=00:15.56
Iter=159, Average Loss=261.791107, Time Elapsed=00:16.01
Iter=160, Average Loss=266.434387, Time Elapsed=00:16.05
Iter=161, Average Loss=262.171021, Time Elapsed=00:16.10
Iter=162, Average Loss=263.764648, Time Elapsed=00:16.14
Iter=163, Average Loss=260.802368, Time Elapsed=00:16.19
Iter=164, Average Loss=271.5886

Iter=291, Average Loss=248.247147, Time Elapsed=00:26.42
Iter=292, Average Loss=248.297913, Time Elapsed=00:26.47
Iter=293, Average Loss=243.892624, Time Elapsed=00:26.52
Iter=294, Average Loss=240.213303, Time Elapsed=00:26.57
Iter=295, Average Loss=245.460632, Time Elapsed=00:27.02
Iter=296, Average Loss=246.202515, Time Elapsed=00:27.07
Iter=297, Average Loss=242.663040, Time Elapsed=00:27.12
Iter=298, Average Loss=241.145844, Time Elapsed=00:27.17
Iter=299, Average Loss=240.247543, Time Elapsed=00:27.22
Iter=300, Average Loss=241.859985, Time Elapsed=00:27.28
Iter=301, Average Loss=245.856567, Time Elapsed=00:27.33
Iter=302, Average Loss=237.746826, Time Elapsed=00:27.39
Iter=303, Average Loss=238.792557, Time Elapsed=00:27.44
Iter=304, Average Loss=239.323929, Time Elapsed=00:27.50
Iter=305, Average Loss=239.649414, Time Elapsed=00:27.56
Iter=306, Average Loss=243.395111, Time Elapsed=00:28.03
Iter=307, Average Loss=247.971237, Time Elapsed=00:28.09
Iter=308, Average Loss=240.9210

Iter=435, Average Loss=231.130051, Time Elapsed=00:40.37
Iter=436, Average Loss=238.998199, Time Elapsed=00:40.42
Iter=437, Average Loss=235.905365, Time Elapsed=00:40.48
Iter=438, Average Loss=233.194000, Time Elapsed=00:40.53
Iter=439, Average Loss=229.319366, Time Elapsed=00:40.59
Iter=440, Average Loss=231.224472, Time Elapsed=00:41.04
Iter=441, Average Loss=229.477768, Time Elapsed=00:41.10
Iter=442, Average Loss=234.943390, Time Elapsed=00:41.15
Iter=443, Average Loss=229.257385, Time Elapsed=00:41.21
Iter=444, Average Loss=233.090302, Time Elapsed=00:41.26
Iter=445, Average Loss=233.243896, Time Elapsed=00:41.32
Iter=446, Average Loss=231.957306, Time Elapsed=00:41.37
Iter=447, Average Loss=232.095749, Time Elapsed=00:41.43
Iter=448, Average Loss=234.358643, Time Elapsed=00:41.48
Iter=449, Average Loss=227.694839, Time Elapsed=00:41.54
Iter=450, Average Loss=226.943665, Time Elapsed=00:41.59
Iter=451, Average Loss=231.729324, Time Elapsed=00:42.05
Iter=452, Average Loss=234.8978

Iter=579, Average Loss=228.027359, Time Elapsed=00:53.56
Iter=580, Average Loss=226.161163, Time Elapsed=00:54.02
Iter=581, Average Loss=225.252792, Time Elapsed=00:54.08
Iter=582, Average Loss=227.118866, Time Elapsed=00:54.13
Iter=583, Average Loss=226.110214, Time Elapsed=00:54.19
Iter=584, Average Loss=227.248688, Time Elapsed=00:54.24
Iter=585, Average Loss=222.226776, Time Elapsed=00:54.30
Iter=586, Average Loss=223.761871, Time Elapsed=00:54.36
Iter=587, Average Loss=222.828720, Time Elapsed=00:54.41
Iter=588, Average Loss=223.399033, Time Elapsed=00:54.47
Iter=589, Average Loss=223.615295, Time Elapsed=00:54.53
Iter=590, Average Loss=223.592209, Time Elapsed=00:54.58
Iter=591, Average Loss=226.668304, Time Elapsed=00:55.05
Iter=592, Average Loss=223.903152, Time Elapsed=00:55.10
Iter=593, Average Loss=226.245117, Time Elapsed=00:55.16
Iter=594, Average Loss=224.010635, Time Elapsed=00:55.21
Iter=595, Average Loss=228.018204, Time Elapsed=00:55.27
Iter=596, Average Loss=225.5411

...

Iter=1721, Average Loss=197.285141, Time Elapsed=02:51.10
Iter=1722, Average Loss=197.177338, Time Elapsed=02:51.17
Iter=1723, Average Loss=195.704391, Time Elapsed=02:51.25
Iter=1724, Average Loss=191.165939, Time Elapsed=02:51.32
Iter=1725, Average Loss=190.240082, Time Elapsed=02:51.39
Iter=1726, Average Loss=191.980240, Time Elapsed=02:51.46
Iter=1727, Average Loss=192.551819, Time Elapsed=02:51.54
Iter=1728, Average Loss=189.899734, Time Elapsed=02:52.02
Iter=1729, Average Loss=186.932022, Time Elapsed=02:52.09
Iter=1730, Average Loss=193.541824, Time Elapsed=02:52.17
Iter=1731, Average Loss=194.278778, Time Elapsed=02:52.24
Iter=1732, Average Loss=201.178177, Time Elapsed=02:52.31
Iter=1733, Average Loss=201.166336, Time Elapsed=02:52.44
Iter=1734, Average Loss=196.881622, Time Elapsed=02:52.51
Iter=1735, Average Loss=193.463928, Time Elapsed=02:52.58
Iter=1736, Average Loss=190.995026, Time Elapsed=02:53.05
Iter=1737, Average Loss=191.970673, Time Elapsed=02:53.12
Iter=1738, Ave

Iter=1863, Average Loss=203.178253, Time Elapsed=03:09.07
Iter=1864, Average Loss=195.246552, Time Elapsed=03:09.15
Iter=1865, Average Loss=195.496140, Time Elapsed=03:09.23
Iter=1866, Average Loss=184.387970, Time Elapsed=03:09.30
Iter=1867, Average Loss=189.250778, Time Elapsed=03:09.38
Iter=1868, Average Loss=187.349762, Time Elapsed=03:09.45
Iter=1869, Average Loss=183.486877, Time Elapsed=03:09.53
Iter=1870, Average Loss=190.669388, Time Elapsed=03:10.01
Iter=1871, Average Loss=183.108307, Time Elapsed=03:10.11
Iter=1872, Average Loss=187.978165, Time Elapsed=03:10.18
Iter=1873, Average Loss=187.874023, Time Elapsed=03:10.26
Iter=1874, Average Loss=187.509323, Time Elapsed=03:10.34
Iter=1875, Average Loss=192.612213, Time Elapsed=03:10.42
Iter=1876, Average Loss=196.937286, Time Elapsed=03:10.50
Iter=1877, Average Loss=198.772781, Time Elapsed=03:10.58
Iter=1878, Average Loss=194.291626, Time Elapsed=03:11.06
Iter=1879, Average Loss=197.030090, Time Elapsed=03:11.17
Iter=1880, Ave

Iter=2005, Average Loss=185.550720, Time Elapsed=03:28.08
Iter=2006, Average Loss=181.823685, Time Elapsed=03:28.15
Iter=2007, Average Loss=181.256104, Time Elapsed=03:28.24
Iter=2008, Average Loss=183.269974, Time Elapsed=03:28.32
Iter=2009, Average Loss=182.224335, Time Elapsed=03:28.40
Iter=2010, Average Loss=183.821442, Time Elapsed=03:28.49
Iter=2011, Average Loss=182.833496, Time Elapsed=03:28.57
Iter=2012, Average Loss=187.837372, Time Elapsed=03:29.06
Iter=2013, Average Loss=185.612976, Time Elapsed=03:29.15
Iter=2014, Average Loss=187.714844, Time Elapsed=03:29.23
Iter=2015, Average Loss=196.472183, Time Elapsed=03:29.32
Iter=2016, Average Loss=196.429749, Time Elapsed=03:29.40
Iter=2017, Average Loss=209.615341, Time Elapsed=03:29.48
Iter=2018, Average Loss=199.589996, Time Elapsed=03:29.57
Iter=2019, Average Loss=195.185333, Time Elapsed=03:30.06
Iter=2020, Average Loss=182.336411, Time Elapsed=03:30.14
Iter=2021, Average Loss=183.931610, Time Elapsed=03:30.23
Iter=2022, Ave

Iter=2147, Average Loss=190.484085, Time Elapsed=03:49.07
Iter=2148, Average Loss=183.505630, Time Elapsed=03:49.15
Iter=2149, Average Loss=183.528046, Time Elapsed=03:49.24
Iter=2150, Average Loss=182.021149, Time Elapsed=03:49.32
Iter=2151, Average Loss=180.229980, Time Elapsed=03:49.40
Iter=2152, Average Loss=179.820343, Time Elapsed=03:49.47
Iter=2153, Average Loss=180.565002, Time Elapsed=03:49.55
Iter=2154, Average Loss=183.050125, Time Elapsed=03:50.03
Iter=2155, Average Loss=181.430450, Time Elapsed=03:50.11
Iter=2156, Average Loss=180.926270, Time Elapsed=03:50.19
Iter=2157, Average Loss=180.443985, Time Elapsed=03:50.28
Iter=2158, Average Loss=182.322708, Time Elapsed=03:50.37
Iter=2159, Average Loss=176.615082, Time Elapsed=03:50.45
Iter=2160, Average Loss=182.325882, Time Elapsed=03:50.53


KeyboardInterrupt: 

## Generate music from trained model

In [131]:
N_GENERATED_NOTE_VECTORS = 1000

In [132]:
outputs_generation, states_generation = rnn.static_rnn(rnn_cell, [x_generation], dtype=tf.float32)
outputs_generation

[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/cell_1/basic_lstm_cell/mul_155:0' shape=(1, 512) dtype=float32>]

In [133]:
logits_generation = tf.nn.xw_plus_b(outputs_generation[0], softmax_w, softmax_b)
logits_generation

<tf.Tensor 'xw_plus_b_51:0' shape=(1, 1124) dtype=float32>

In [177]:
#seed_note = np.zeros(vocab_size)
#seed_note[0] = 1
seed_note = training_data[20][0]
seed_note = np.reshape(seed_note, [1, vocab_size])

In [162]:
generated_music = [seed_note]
for i in range(N_GENERATED_NOTE_VECTORS):
    onehot_pred = session.run(logits_generation, feed_dict={x_generation: generated_music[-1]})
    
    next_note = np.zeros(vocab_size)
    #index = np.argmax(onehot_pred, 1) # take max probability
    softmax_cdf = np.exp(onehot_pred[0])/np.sum(np.exp(onehot_pred[0]))
    index = np.random.choice(range(vocab_size), p=softmax_cdf) # choose probabilistically
    
    next_note[index] = 1
    next_note = np.reshape(next_note, [1, vocab_size])
    
    generated_music.append(next_note)

In [188]:
notes = np.array(generated_music).reshape(N_GENERATED_NOTE_VECTORS+1, 1124).dot(np.array(range(1124)))
notes

array([ 1047.,   196.,   321., ...,  1024.,   552.,  1030.])

# Convert generated music to midi

In [180]:
def get_track_sequences(vector_sequence):
    """Get note sequences for each track from vector sequence output from RNN.

    Args:
       vector_sequence: List of one-hot vectors containing 128 * NUM_TRACKS note_on events, 128 * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 10 ms each.
       
    Returns:
        List of track vector sequences.
    """
    track_sequences = [[] for _ in range(NUM_TRACKS)]
    start_time = 0
    for vector in vector_sequence:
        index = vector.index(1)
        # vector is a time event.
        if index >= NUM_MIDI_PITCHES * 2 * NUM_TRACKS:
            time = (index - NUM_MIDI_PITCHES * 2 * NUM_TRACKS + 1) * TIMESHIFT_LENGTH
            start_time += time
        else:
            # vector is a note_off event.
            if index >= NUM_MIDI_PITCHES * NUM_TRACKS:
                note_type = 'note_off'
                index -= NUM_MIDI_PITCHES * NUM_TRACKS
            # vector is a note_on event.
            else:
                note_type = 'note_on'
            track_num = int(index / NUM_MIDI_PITCHES)
            note = index % NUM_MIDI_PITCHES
            track_sequences[track_num].append({"type": note_type, "note": note, "start_time": start_time})
            time_delay = 0
            
    return track_sequences
    

In [181]:
def get_midi_from_vector_sequence(track_sequences):
    """Get midi from track sequences.

    Args:
       track_sequences: List of vector sequences for each track.
       
    Returns:
        MidiFile object.
    """
    mid = mido.MidiFile()
    tracks = [mido.MidiTrack() for _ in range(NUM_TRACKS)]
    mid.tracks.extend(tracks)

    for i, ts in enumerate(track_sequences):
        prev_start_time = 0
        for event in ts:
            time = event["start_time"] - prev_start_time
            prev_start_time = event["start_time"]
            # I picked a random number for velocity.
            mid.tracks[i].append(mido.Message(event["type"], note=event["note"], velocity=50,
                                              time=int(mido.second2tick(time, TICKS_PER_BEAT, TEMPO))))
    
    return mid

In [182]:
track_sequence = get_track_sequences([list(x[0]) for x in generated_music])
track_sequence

[[{'note': 37, 'start_time': 0.24, 'type': 'note_off'},
  {'note': 37, 'start_time': 0.36, 'type': 'note_off'},
  {'note': 41, 'start_time': 0.48, 'type': 'note_off'},
  {'note': 41, 'start_time': 0.6, 'type': 'note_off'},
  {'note': 44, 'start_time': 0.72, 'type': 'note_off'},
  {'note': 44, 'start_time': 0.84, 'type': 'note_off'},
  {'note': 49, 'start_time': 0.96, 'type': 'note_off'},
  {'note': 49, 'start_time': 1.08, 'type': 'note_off'},
  {'note': 41, 'start_time': 1.2000000000000002, 'type': 'note_off'},
  {'note': 41, 'start_time': 1.3200000000000003, 'type': 'note_off'},
  {'note': 40, 'start_time': 1.4400000000000004, 'type': 'note_off'},
  {'note': 40, 'start_time': 1.9500000000000004, 'type': 'note_off'},
  {'note': 40, 'start_time': 1.9800000000000004, 'type': 'note_off'},
  {'note': 45, 'start_time': 2.6900000000000004, 'type': 'note_off'},
  {'note': 40, 'start_time': 3.9, 'type': 'note_off'},
  {'note': 41, 'start_time': 4.9, 'type': 'note_off'},
  {'note': 40, 'start_t

In [183]:
midi = get_midi_from_vector_sequence(track_sequence)
midi

<midi file None type 1, 4 tracks, 483 messages>

# Save midi

In [184]:
midi.save('generated_music.mid')